In [1]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 801.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 38.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=d026538581ad16393bfbb6e2679e9fe3bc4c17f62e4bf31029f203e2d771031f
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.2 MB/s eta 0:00:00


In [3]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import pymorphy2
import nltk
import string

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, classification_report
from wordcloud import WordCloud, STOPWORDS
from catboost import CatBoostClassifier, CatBoostRegressor, Pool
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [4]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
LEMMATIZER = WordNetLemmatizer()
PORTER_STEMMER = PorterStemmer()
ENGLISH_STOP_WORDS = set(stopwords.words('english'))

In [6]:
from google.colab import drive

In [7]:
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
df = pd.read_csv('/content/drive/MyDrive/UrfuMagistracy/natural-language-processing-III/datasets/train.csv', index_col=0)

In [9]:
df.head()

Text           Sentiment
0  @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...             Neutral
1  advice Talk to your neighbours family to excha...            Positive
2  Coronavirus Australia: Woolworths to give elde...            Positive
3  My food stock is not the only one which is emp...            Positive
4  Me, ready to go at supermarket during the #COV...  Extremely Negative

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41159 entries, 0 to 41156
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       41158 non-null  object
 1   Sentiment  41155 non-null  object
dtypes: object(2)
memory usage: 964.7+ KB


In [11]:
df.describe()

Text Sentiment
count                                               41158     41155
unique                                              41158         5
top     @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...  Positive
freq                                                    1     11422

### Preprocessing

In [12]:
df.isna().sum().sum()

5

In [13]:
df.dropna(inplace=True)

In [14]:
df.isna().sum().sum()

0

In [15]:
def text_preprocessing(text, stop_words=None, stem=None, lemmatizer=None):
    text = re.sub(r'http\S+', '', text)

    text = text.translate(str.maketrans("", "", string.punctuation))
    text = ''.join([i for i in text if not i.isdigit()])
    text = text.lower()

    if stop_words is not None:
        words = nltk.word_tokenize(text)
        filtered_words = [word for word in words if word not in stop_words]
        text = " ".join(filtered_words)

    if stem is not None:
        words = nltk.word_tokenize(text)
        stemmed_words = [stem.stem(word) for word in words]
        text = " ".join(stemmed_words)

    if lemmatizer is not None:
        words = nltk.word_tokenize(text)
        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
        text = " ".join(lemmatized_words)

    return text

In [16]:
df['Text'] = df['Text'].apply(text_preprocessing, args=(ENGLISH_STOP_WORDS, PORTER_STEMMER, LEMMATIZER,))

In [17]:
df = df[df['Text'] != '']

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 41139 entries, 0 to 41156
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Text       41139 non-null  object
 1   Sentiment  41139 non-null  object
dtypes: object(2)
memory usage: 964.2+ KB


In [19]:
df.head()

Text           Sentiment
0                         menyrbi philgahan chrisitv             Neutral
1  advic talk neighbour famili exchang phone numb...            Positive
2  coronaviru australia woolworth give elderli di...            Positive
3  food stock one empti plea dont panic enough fo...            Positive
4  readi go supermarket covid outbreak im paranoi...  Extremely Negative

### Naive classification

In [20]:
df_naive = df.copy()

In [21]:
X_train_naive, X_test_naive, y_train_naive, y_test_naive = train_test_split(df_naive.Text, df_naive.Sentiment, shuffle=True, test_size=0.25, random_state=42)

In [22]:
vectorizer = CountVectorizer()

In [23]:
train_features = vectorizer.fit_transform(X_train_naive)
test_features = vectorizer.transform(X_test_naive)

In [24]:
train_features.shape

(30854, 39615)

In [25]:
test_features.shape

(10285, 39615)

In [26]:
classifier = MultinomialNB()

In [27]:
classifier.fit(train_features, y_train_naive)

MultinomialNB()

In [28]:
predictions = classifier.predict(test_features)

In [29]:
print(classification_report(y_test_naive, predictions))

                    precision    recall  f1-score   support

Extremely Negative       0.64      0.30      0.41      1394
Extremely Positive       0.63      0.37      0.47      1669
          Negative       0.41      0.54      0.46      2439
           Neutral       0.69      0.30      0.42      1948
          Positive       0.40      0.65      0.50      2835

          accuracy                           0.47     10285
         macro avg       0.56      0.43      0.45     10285
      weighted avg       0.53      0.47      0.46     10285



### Split

In [30]:
vectorizer = CountVectorizer()
X_train_numeric = vectorizer.fit_transform(df.Text)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X_train_numeric, df.Sentiment, shuffle=True, test_size=0.25, random_state=42)

In [32]:
X_train.shape

(30854, 48083)

In [33]:
y_train.shape

(30854,)

In [34]:
y_test.shape

(10285,)

In [35]:
X_train_pool = Pool(data=X_train, label=y_train, cat_features=[])

In [36]:
X_evaluation_pool = Pool(data=X_test, label=y_test, cat_features=[])

In [37]:
model = CatBoostClassifier(iterations=1000, early_stopping_rounds=100)

In [38]:
model.fit(X=X_train_pool, eval_set=X_evaluation_pool)

Learning rate set to 0.115642
0:	learn: 1.5811319	test: 1.5816213	best: 1.5816213 (0)	total: 646ms	remaining: 10m 45s
1:	learn: 1.5567409	test: 1.5573827	best: 1.5573827 (1)	total: 1.14s	remaining: 9m 27s
2:	learn: 1.5390397	test: 1.5400349	best: 1.5400349 (2)	total: 1.9s	remaining: 10m 32s
3:	learn: 1.5238802	test: 1.5246053	best: 1.5246053 (3)	total: 2.71s	remaining: 11m 14s
4:	learn: 1.5088519	test: 1.5104393	best: 1.5104393 (4)	total: 3.51s	remaining: 11m 37s
5:	learn: 1.4979277	test: 1.4990436	best: 1.4990436 (5)	total: 4.32s	remaining: 11m 56s
6:	learn: 1.4875981	test: 1.4895932	best: 1.4895932 (6)	total: 5.11s	remaining: 12m 4s
7:	learn: 1.4780028	test: 1.4803751	best: 1.4803751 (7)	total: 5.85s	remaining: 12m 5s
8:	learn: 1.4699273	test: 1.4721778	best: 1.4721778 (8)	total: 6.64s	remaining: 12m 11s
9:	learn: 1.4618692	test: 1.4646983	best: 1.4646983 (9)	total: 7.38s	remaining: 12m 10s
10:	learn: 1.4557470	test: 1.4587245	best: 1.4587245 (10)	total: 7.92s	remaining: 11m 52s
11:	

In [39]:
y_pred = model.predict(X_test)

In [40]:
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.6201263976665046


In [41]:
print(f"Precision: {precision_score(y_test, y_pred, average='weighted')}")

Precision: 0.6306122450634108


In [42]:
print(f"Recall: {recall_score(y_test, y_pred, average='weighted')}")

Recall: 0.6201263976665046
